In [1]:
%matplotlib inline
import SimpleITK as sitk
import os
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, fixed
from IPython.display import clear_output
from datasets.syntheticCT import create_registration_mask


## Help functions

In [10]:
def display_image_planes(image_x, image_y, image_z, image):
    fig = plt.gcf()
    fig.set_size_inches(30/2.54, 15/2.54)
    ax1 = fig.add_subplot(131)
    ax2 = fig.add_subplot(132)
    ax3 = fig.add_subplot(133)
    
    img_x = image[image_x,:,:]
    ax1.imshow(sitk.GetArrayViewFromImage(img_x), cmap=plt.cm.Greys_r)
    #ax1.axis("off")
    ax1.set_title('X plane')
    
    img_y = image[:, image_y,:]
    ax2.imshow(sitk.GetArrayViewFromImage(img_y), cmap=plt.cm.Greys_r)
    ax2.axis("off")
    ax2.set_title('Y plane')
    
    img_z = image[:, :, image_z]
    ax3.imshow(sitk.GetArrayViewFromImage(img_z), cmap=plt.cm.Greys_r)
    ax3.axis("off")
    ax3.set_title('Z plane')
    
    fig.tight_layout()
    plt.show()
    
def display_images_planes_with_alpha(image_x, image_y, image_z, alpha, fixed, moving):
    fig = plt.gcf()
    fig.set_size_inches(60/2.54, 30/2.54)
    ax1 = fig.add_axes([0., 0.55, 0.60, 0.55])
    ax2 = fig.add_axes([0., 0., 0.60, 0.55])
    ax3 = fig.add_axes([0.65, 0., 0.35, 1])
    
    img_x = (1.0 - alpha) * fixed[image_x,:,:] + alpha * moving[image_x,:,:]
    ax1.imshow(sitk.GetArrayViewFromImage(img_x), cmap=plt.cm.Greys_r)
    ax1.axis("off")
    ax1.set_title('X plane', fontsize=24)
    
    img_y = (1.0 - alpha) * fixed[:, image_y,:] + alpha * moving[:, image_y,:]
    ax2.imshow(sitk.GetArrayViewFromImage(img_y), cmap=plt.cm.Greys_r)
    ax2.axis("off")
    ax2.set_title('Y plane', fontsize=24)
    
    img_z = (1.0 - alpha) * fixed[:, :, image_z] + alpha * moving[:, :, image_z]
    ax3.imshow(sitk.GetArrayViewFromImage(img_z), cmap=plt.cm.Greys_r)
    ax3.axis("off")
    ax3.set_title('Z plane', fontsize=24)
    
    plt.show()


def rigid_translation(image_x, image_y, image_z, alpha, tx, ty, tz, fixed, moving):
    
    # resample moving image using tx, ty, tz
    translation_transform = sitk.TranslationTransform(3)
    translation_transform.SetParameters((tx, ty, tz))
    
    moving_translated = sitk.Resample(
        moving,
        fixed,
        translation_transform,
        sitk.sitkLinear,
        -1000,
        fixed.GetPixelID(),
    )
    
        
    fig = plt.gcf()
    fig.set_size_inches(60/2.54, 40/2.54)
    ax1 = fig.add_axes([0., 0.55, 0.60, 0.55])
    ax2 = fig.add_axes([0., 0., 0.60, 0.55])
    ax3 = fig.add_axes([0.65, 0., 0.35, 1])
    
    img_x = (1.0 - alpha) * fixed[image_x,:,:] + alpha * moving_translated[image_x,:,:]
    ax1.imshow(sitk.GetArrayViewFromImage(img_x), cmap=plt.cm.Greys_r)
    ax1.axis("off")
    ax1.set_title('X plane ({}), Tx = {} mm'.format(image_x, tx), fontsize=24)
    
    img_y = (1.0 - alpha) * fixed[:, image_y,:] + alpha * moving_translated[:, image_y,:]
    ax2.imshow(sitk.GetArrayViewFromImage(img_y), cmap=plt.cm.Greys_r)
    ax2.axis("off")
    ax2.set_title('Y plane ({}), Ty = {} mm'.format(image_y, ty), fontsize=24)
    
    img_z = (1.0 - alpha) * fixed[:, :, image_z] + alpha * moving_translated[:, :, image_z]
    ax3.imshow(sitk.GetArrayViewFromImage(img_z), cmap=plt.cm.Greys_r)
    ax3.axis("off")
    ax3.set_title('Z plane ({}), Tz = {} mm'.format(image_z, tz), fontsize=24)
    
    plt.show()
    

def output_image(image, filename):
    writer = sitk.ImageFileWriter()
    writer.SetFileName(filename)
    writer.SetImageIO("NiftiImageIO")
    writer.Execute(image)


## Read image data

In [16]:
patient_id = '22'
fraction = 'F1'

root = r'C:\temp\5-days-recti\PatientData'
result_path = os.path.join(root, '{}_anonymized'.format(patient_id), 'synthetic_CT', fraction)

print('start reading images')
ct_ref_float = sitk.ReadImage(os.path.join(result_path, 'ct_ref.nii'))
print('start reading images 1')
cbct_float = sitk.ReadImage(os.path.join(result_path, 'cbct.nii'))
cbct_matched = sitk.ReadImage(os.path.join(result_path, 'cbct_matched.nii'))
cbct_in_ct_for = sitk.ReadImage(os.path.join(result_path, 'cbct_in_ct_for.nii'))
cbct_cog_in_ct_for = sitk.ReadImage(os.path.join(result_path, 'cbct_cog.nii'))


interact(
    display_image_planes,
    image_x=(0, cbct_float.GetSize()[0] - 1),
    image_y=(0, cbct_float.GetSize()[1] - 1),
    image_z=(0, cbct_float.GetSize()[2] - 1),
    image=fixed(cbct_float),
);

 
    


start reading images
start reading images 1


interactive(children=(IntSlider(value=134, description='image_x', max=269), IntSlider(value=134, description='…

## Evaluate original rigid registration results

In [17]:
import ipywidgets

wx = ipywidgets.IntSlider(value = ct_ref_float.GetSize()[0] // 2, min = 0, max = ct_ref_float.GetSize()[0] - 1, description='X')
wy = ipywidgets.IntSlider(value = ct_ref_float.GetSize()[1] // 2, min = 0, max = ct_ref_float.GetSize()[1] - 1, description='Y')
wz = ipywidgets.IntSlider(value = ct_ref_float.GetSize()[2] // 2, min = 0, max = ct_ref_float.GetSize()[2] - 1, description='Z')
walpha = ipywidgets.FloatSlider(value = 0.5, min = 0., max = 1., description='Alpha')
ui = ipywidgets.VBox([ipywidgets.HBox([wx, wy, wz]), ipywidgets.HBox([walpha])])

out = ipywidgets.interactive_output(display_images_planes_with_alpha, 
                                    {'image_x': wx, 'image_y': wy, 'image_z': wz, 'alpha': walpha, 
                                     'fixed' : fixed(ct_ref_float), 'moving' : fixed(cbct_in_ct_for)})

display(ui, out)



Output()

## Manual rigid translation

In [18]:
import ipywidgets

wx = ipywidgets.IntSlider(value = ct_ref_float.GetSize()[0] // 2, min = 0, max = ct_ref_float.GetSize()[0] - 1, description='X')
wy = ipywidgets.IntSlider(value = ct_ref_float.GetSize()[1] // 2, min = 0, max = ct_ref_float.GetSize()[1] - 1, description='Y')
wz = ipywidgets.IntSlider(value = ct_ref_float.GetSize()[2] // 2, min = 0, max = ct_ref_float.GetSize()[2] - 1, description='Z')
walpha = ipywidgets.FloatSlider(value = 0.5, min = 0., max = 1., step = 0.1, description='Alpha')

tx = ipywidgets.FloatSlider(value = 0, min = -50., max = 50., step = 0.5, description = 'Trans X (mm)')
ty = ipywidgets.FloatSlider(value = 0, min = -50., max = 50., step = 0.5, description = 'Trans Y (mm)')
tz = ipywidgets.FloatSlider(value = 0, min = -50., max = 50., step = 0.5, description = 'Trans Z (mm)')

ui = ipywidgets.VBox([ipywidgets.HBox([wx, wy, wz]), ipywidgets.HBox([walpha]), ipywidgets.HBox([tx, ty, tz])])


out = ipywidgets.interactive_output(rigid_translation, 
                                    {'image_x': wx, 'image_y': wy, 'image_z': wz, 'alpha': walpha,
                                     'tx': tx, 'ty': ty, 'tz': tz, 
                                     'fixed' : fixed(ct_ref_float), 'moving' : fixed(cbct_in_ct_for)})

display(ui, out)



Output()

## Save the resample image from the manual registration 

In [19]:
# translation from manual rigid registration
translation = (tx.get_interact_value(), ty.get_interact_value(), tz.get_interact_value())
print('Manual translation:', translation)
               
translation_transform = sitk.TranslationTransform(3)
translation_transform.SetParameters(translation)

cbct_in_ct_for_manual = sitk.Resample(
    cbct_in_ct_for,
    ct_ref_float,
    translation_transform,
    sitk.sitkLinear,
    -1000,
    ct_ref_float.GetPixelID(),
)
               
output_image(cbct_in_ct_for_manual, os.path.join(result_path, 'cbct_in_ct_for_manual.nii'))
               

Manual translation: (0.0, 0.0, 12.5)


## Double Check results by reading the new image

In [20]:
cbct_in_ct_for_manual_read = sitk.ReadImage(os.path.join(result_path, 'cbct_in_ct_for_manual.nii'))

wx = ipywidgets.IntSlider(value = ct_ref_float.GetSize()[0] // 2, min = 0, max = ct_ref_float.GetSize()[0] - 1, description='X')
wy = ipywidgets.IntSlider(value = ct_ref_float.GetSize()[1] // 2, min = 0, max = ct_ref_float.GetSize()[1] - 1, description='Y')
wz = ipywidgets.IntSlider(value = ct_ref_float.GetSize()[2] // 2, min = 0, max = ct_ref_float.GetSize()[2] - 1, description='Z')
walpha = ipywidgets.FloatSlider(value = 0.5, min = 0., max = 1., description='Alpha')
ui = ipywidgets.VBox([ipywidgets.HBox([wx, wy, wz]), ipywidgets.HBox([walpha])])

out = ipywidgets.interactive_output(display_images_planes_with_alpha, 
                                    {'image_x': wx, 'image_y': wy, 'image_z': wz, 'alpha': walpha, 
                                     'fixed' : fixed(ct_ref_float), 'moving' : fixed(cbct_in_ct_for_manual_read)})

display(ui, out)


Output()